# 파이토치(PyTorch) 기초

1. 텐서 생성
2. 텐서 다루기
3. 손실 함수

## 텐서 생성

In [48]:
import torch
import numpy as np

# NumPy 배열로부터 텐서 생성
data_numpy = np.array(
    [
        [
            [1, 2],
            [3, 4], 
            [5, 6]
        ],
        [
            [11, 12],
            [13, 14], 
            [15, 16]
        ]
    ]
)

# torch.tensor()는 데이터를 복사하여 새로운 텐서를 만듭니다.
tensor_from_numpy_copy = torch.tensor(data_numpy)
print(tensor_from_numpy_copy)
print(tensor_from_numpy_copy.size())

tensor([[[ 1,  2],
         [ 3,  4],
         [ 5,  6]],

        [[11, 12],
         [13, 14],
         [15, 16]]])
torch.Size([2, 3, 2])


In [49]:
# 0과 1 사이의 균등 분포(uniform distribution)에서 랜덤 값 생성
shape = (2, 3, 2)
data_random = torch.rand(shape)
print(data_random)
print(data_random.size())

tensor([[[0.1053, 0.2695],
         [0.3588, 0.1994],
         [0.5472, 0.0062]],

        [[0.9516, 0.0753],
         [0.8860, 0.5832],
         [0.3376, 0.8090]]])
torch.Size([2, 3, 2])


## 텐서 다루기

In [50]:
t1, t2 = tensor_from_numpy_copy.split(1, dim=-1)
print(t1)
print(t2)
print(t2.size())

tensor([[[ 1],
         [ 3],
         [ 5]],

        [[11],
         [13],
         [15]]])
tensor([[[ 2],
         [ 4],
         [ 6]],

        [[12],
         [14],
         [16]]])
torch.Size([2, 3, 1])


In [51]:
t1 = t1.squeeze(-1).contiguous()
t2 = t2.squeeze(-1).contiguous()
print(t1)
print(t2)
print(t2.size())

tensor([[ 1,  3,  5],
        [11, 13, 15]])
tensor([[ 2,  4,  6],
        [12, 14, 16]])
torch.Size([2, 3])


In [52]:
# 차원
print(t1.size())
print(len(t1.size()))
print(t1.size(1))

torch.Size([2, 3])
2
3


In [53]:
start_positions = torch.tensor([1, 2, 3, 4])
print(start_positions)
start_positions = start_positions.clamp(0, 2)
print(start_positions)

tensor([1, 2, 3, 4])
tensor([1, 2, 2, 2])


## 손실 함수

문장 분류 문제 유형 및 손실 함수

| 문제 유형 | 할당 값 | 손실 함수 |
| -- | -- | -- |
| 회귀 (Regression) | 연속적인 수치 값 | [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html) |
| 단일 레이블 분류 (Single-Label Classification) | 상호 배타적인 범주 중 하나 | [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) |
| 다중 레이블 분류 (Multi-Label Classification) | 상호 배타적이지 않은 범주 중 여러 개 (0개 포함) | [nn.BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html), [nn.BCEWithLogitsLoss](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html) |

### MSELoss

In [61]:
torch.manual_seed(42)

loss = nn.MSELoss()
input = torch.randn(3, 5)
target = torch.randn(3, 5)
output = loss(input, target)
print(output.item())

1.0191929340362549


### BCELoss

In [62]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, 2)
target = torch.rand(3, 2)
print(input)
print(target)
output = loss(m(input), target)
print(output.item())

tensor([[-0.0085,  0.7291],
        [ 0.1331,  0.8640],
        [-1.0157, -0.8887]])
tensor([[0.9811, 0.0874],
        [0.0041, 0.1088],
        [0.1637, 0.7025]])
0.8472294807434082


### BCEWithLogitsLoss

In [63]:
loss = nn.BCEWithLogitsLoss()
input = torch.randn(3)
target = torch.empty(3).random_(2)
print(input)
print(target)
output = loss(input, target)
print(output.item())

tensor([-2.1256,  0.9629,  0.7596])
tensor([1., 0., 1.])
1.302810549736023


### CrossEntropyLoss

In [56]:
from torch import nn

# Example of target with class indices

batch_size = 3
max_seq_length = 4

start_positions = torch.tensor([0, 1, 4]) # (batch_size,)
end_positions = torch.tensor([1, 2, 4]) # (batch_size,)

start_logits = torch.rand((batch_size, max_seq_length)) # (batch_size, max_seq_length)
end_logits = torch.rand((batch_size, max_seq_length)) # (batch_size, max_seq_length)

ignored_index = start_logits.size(1)
print(f"ignored_index: {ignored_index}")
start_positions = start_positions.clamp(0, ignored_index)
end_positions = end_positions.clamp(0, ignored_index)

loss_fct = nn.CrossEntropyLoss(ignore_index=ignored_index)
start_loss = loss_fct(start_logits, start_positions)
end_loss = loss_fct(end_logits, end_positions)
total_loss = (start_loss + end_loss) / 2
print(f"start_loss: {start_loss.item():.4f}, end_loss: {end_loss.item():.4f}, total_loss: {total_loss.item():.4f}")

ignored_index: 4
start_loss: 1.3153, end_loss: 1.6081, total_loss: 1.4617
